In [10]:
import requests
import urllib
from bs4 import BeautifulSoup
import re
import io
import gzip
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import glob
import os
from os import path
import datetime
from dateutil.parser import parse
from selenium.webdriver.support.ui import Select


In [11]:
def security_numbers():
    df_equity = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Equity.csv")
    security_numbers = df_equity["Security Code"].tolist()
    return security_numbers

In [12]:
def get_security_numbers():
    
    p = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data"
    dir_list = os.listdir(p)

    if ("Equity.csv" in dir_list):
        print("Equity already present")
        sn = security_numbers()
        return sn
    else:
        sn = get_equity()
        return sn
    

In [13]:
def get_equity():
    
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_experimental_option("prefs", {"download.default_directory": "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data"})

    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)

    url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
    driver.get(url)

    segment = "Equity"
    status = "Active"

    segment_element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddSegment"]')
    segment_element.send_keys(segment)

    status_element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_ddlStatus"]')
    status_element.send_keys(status)

    submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
    submit.send_keys(Keys.RETURN)


    download = driver.find_element_by_xpath(
        '/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i')
    download.click()
    time.sleep(5)
    driver.quit()
    sec_num = security_numbers()
    return sec_num

In [14]:
def get_date(d,m,y, driver):
    yy = 2021
    while (int(y) < yy):
        yy -= 10
        year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]')
        segment = Select(year)
        segment.select_by_visible_text(str(yy))
    year = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[2]')
    segment = Select(year)
    segment.select_by_visible_text(y)

    month = driver.find_element_by_xpath('/html/body/div[1]/div/div/select[1]')
    segment = Select(month)
    segment.select_by_visible_text(m)

    days = driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
    days.click()

In [15]:
def traverse_stock(security, driver):
    if (path.exists("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\" + security + ".csv")):
        print("present")
        df = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\" + security + ".csv")
        fr_df = df.tail(1)['Date']
        fr_date = fr_df.iloc[0]
        f = parse(fr_date)
        fd = f.date()
        if (fd == (datetime.datetime.today() - datetime.timedelta(days=1)).date()):
            return None
        else:
            print("not current date")
            months = {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 
                     7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
            day = 1 if (fd.day + 1 == 32) else fd.day + 1
            month = -1
            if (fd.day + 1 == 32):
                month = fd.month + 1
                if (month == 13):
                    month = 1
            else:
                month = fd.month
            year = -1
            if (fd.day + 1 == 32 and fd.month + 1 == 13):
                year = fd.year + 1
            else:
                year = fd.year
            element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_smartSearch"]')
            element.clear()
            element.send_keys(security)
            element.send_keys(Keys.ENTER)
            time.sleep(2)
            from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtFromDate"]')
            from_date.clear()
            from_date.click()
            get_date(str(day),months[month],str(year), driver)
            to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtToDate"]')
            to_date.clear()
            to_date.click()
            cd = datetime.datetime.today()
            get_date(str(cd.day),months[cd.month],str(cd.year), driver)
            submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
            submit.click()
            time.sleep(2)
            store_stock(security, driver)
            return None


    element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_smartSearch"]')
    element.clear()
    element.send_keys(security)
    element.send_keys(Keys.ENTER)
    time.sleep(2)
    try:
        response = driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[3]/div/div/table/tbody/tr[2]/td/div/div[2]/div/div/div[2]/div[1]/table/tbody/tr/td/div/div/div/ul/li")
    except Exception as e:
        pass

    if response.text == "No Match Found":
        return None
    else:
        print("Not present")
        months = {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 
                 7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
        from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtFromDate"]')
        from_date.clear()
        from_date.click()
        get_date("01","Jan","2007", driver)
        to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtToDate"]')
        to_date.clear()
        to_date.click()
#             cd = datetime.datetime.today()
#             get_date(str(cd.day),months[cd.month],str(cd.year))
        get_date("31", "Dec", "2020", driver)
        submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
        submit.click()
        time.sleep(5)
        store_stock(security, driver)

In [16]:
def remake_stock(security):
    files_list = os.listdir("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data")
    if (security +  " (1).csv") not in files_list:
        path = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\" + security + ".csv"
        stock = pd.read_csv(path)
        fr_df = stock.tail(1)['Date']
        fr_date = fr_df.iloc[0]
        f = parse(fr_date)
        fd = f.date()
        if (fd == (datetime.datetime.today() - datetime.timedelta(days=1)).date()):
            return None
        stock = stock[::-1]
        stock.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\" + security + ".csv", index=False)
    else:
        path = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\" + security + " (1).csv"
        stock = pd.read_csv(path)
        stock = stock[::-1]
        stock.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\" + security + " (1).csv", index=False)
        merge_files = []
        for file in files_list:
            if file.startswith(security):
                merge_files.append(file)
        pth = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\"
        combined_csv = pd.concat([pd.read_csv(pth + merge_files[f]) for f in range(len(merge_files) - 1, -1, -1)])
        combined_csv.to_csv( pth + security + ".csv", index=False, encoding='utf-8-sig')
        os.remove(path)


In [17]:
def store_stock(security, driver):
    try:
        download_stock = driver.find_element_by_xpath("/html/body/form/div[4]/div/div/div[1]/div/div[4]/div/div/div[1]/div[4]/div[2]/div/span/a/i")
        download_stock.click()
        time.sleep(5)
        remake_stock(security)
    except Exception as e:
        pass

In [18]:
def get_stocks(nums):
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_experimental_option("prefs", {"download.default_directory": "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data"})
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)
    url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0"
    driver.get(url)
    traverse_stock(str(nums), driver)
#     time.sleep(5)
    driver.quit()
#     remake_stock(str(nums))
